# Laboratório 01 - fast.ai DL1 2019 Brasília


### Introdução

Bem vindos ao primeiro laboratório da turma fast.ai DL1 2019. O objetivo do laboratório é consolidar os conhecimentos obtidos nas aulas 01 e 02, bem como promover debate, discussões relacionadas ao tema, bem como fazer com que a maioria coloque a "mão na massa" como gosta nosso guru, Jeremy.

Realizaremos nesse laboratório o treinamento de uma rede neural para classificação de expressões faciais baseadas em fotos de pessoas. Utilizaremos a base de dados de um desafio do Kaggle de 2013, conforme detalhes mais adiante. O modelo treinado terá alcançado uma acurácia similiar, ou até superior, aos melhores modelos da competição em 2013. Além disso, vamos desenvolver um aplicativo web simples para colocar em produção nosso modelo treinado de a nos permitir testar novas imagens com expressões a avaliar a acurácia da rede neural na prática.

Esse notebook é uma combinação de dois posts do colega [Pierre Guillou](https://www.linkedin.com/in/pierreguillou/), que cedeu seu código para que possamos consolidar nossos conhecimentos:

- [Face Expression Recognition with fastai v1](https://medium.com/@pierre_guillou/face-expression-recognition-with-fastai-v1-dc4cf6b141a3).
- [Deep Learning Web App by fastai v1](https://medium.com/@pierre_guillou/deep-learning-web-app-by-fastai-v1-3ab4c20b7cac)


### Reconhecimento de Expressões Faciais (Face Expression Recognition Kaggle Challenge - FER 2013) com fastai v1

by: [Pierre Guillou](https://www.linkedin.com/in/pierreguillou/) (novembro 2018)  
    [Leon Silva](https://www.linkedin.com/in/leonsolon/) (tradução/atualização março 2019)
    [Vinícius Ramos](https://www.linkedin.com/in/vinicius-ramos-6a367073/) (atualização/deploy no Django na GCP março 2018)

- Inspirado por [Recognizing Facial Expressions Using Deep Learning](http://cs231n.stanford.edu/reports/2017/pdfs/224.pdf) (2017) of Alexandru Savoiu and James Wong (Stanford University)
- Base de dados : [FER 2013 Kaggle Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/)
- Deep Learning utilizando [fastai v1](https://www.fast.ai/2018/10/02/fastai-ai/)

# Inicialização

Vamos começar como todo e qualquer notebook que utiliza fast.ai

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *

# Configuração da API Kaggle

Faremos o download do banco de imagens diretamente do Kaggle. Para tal, é necessário seguir os passos abaixo. Este notebook está focalizado na google cloud platform (GCP), tendo em vista que a maioria da turma realizou essa instalação. Se você está lendo isso é porque já está rodando o jupyter de alguma forma, seja na máquina local, seja no gcp, portanto, pularemos os passos de inicialização da instância e do jupyter.

1. Caso ainda não possua, cria sua conta no [Kaggle](https://www.kaggle.com) no seu computador local
2. No canto superior direito, clique na sua foto (ou avatar) e selecione 'My account'
3. Selecione a opção 'Create API Token'. 
4. Ao selecionar a opção será baixado um arquivo na sua máquina, chamada `kaggle.json`. Esse arquivo contém as credenciais da API do Kaggle (nome de usuário e senha)
5. Abra o arquivo `kaggle.json` no seu editor de textos preferido e copie o conteúdo. Cole o conteúdo na célula abaixo, no local indicado.

In [ ]:
# Comando que cria a pasta Kaggle em /home/<seu_usuario>. Essa pasta é necessária para conter a chave de API do Kaggle
!mkdir ~/.kaggle

# Esse comando cria o arquivo kaggle.json na instância no local correto: /home/<seu_usuario/.kaggle/kaggle.json
!echo 'coloque o conteudo do kaggle.json aqui' >> ~/.kaggle/kaggle.json

# Por questões de segurança, garanta que outros usuários do seu computador não tenham acesso de leitura 
# às suas credenciais. Esse comando é necessário, do contrário a API do Kaggle se recusa a fazer o download das bases de dados.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Com a chave de API do Kaggle, agora é a vez de instalar a API do kaggle
!sudo pip install kaggle

### Termos da competição

Um passo importante, antes de baixar os dados do Kaggle, é aceitar os termos da competição. Para isso, acesse a seção `rules` na [página da competição](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/rules) e clique em `I Understand and Accept`

# Dados

## Download do Kaggle

Agora que temos a conta, chave de API e pacote instalado, basta realizarmos o download das imagens com as expressões faciais direto do kaggle. 

In [ ]:
# Comando para alterar pasta para '~/.fastai/data', onde os arquivos serão armazenados. 
%cd ~/.fastai/data/

# Finnalmente, download do arquivo FER 2013 a partir da conta do Kaggle (fer2013.tar.gz)
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
# Use os comandos abaixo para criar uma pasta para FER (face expression recognition)
# e para descompactar o arquivo dentro da pasta FER
!mkdir FER
!mv fer2013.tar.gz FER
!rm example_submission.csv
!tar -xvf FER/fer2013.tar.gz -C FER

## Listando arquivos

In [ ]:
# Verificar se os arquivos foram decompactados corretamente
%cd ~/.fastai/data/
PATH = Path('./FER/fer2013')
PATH.ls()

In [ ]:
# Nessa competição, as imagens foram armazenadas em forma de pixels, com os valores RGB de cada
# A 1a coluna apresenta a emoção (expressão) a qual a imagem representa
# A 2a coluna apresenta a imagem em si, num arranjo de números RFB que representa os pixels
# A 3a coluna traz a utilidade da imagem, se será utilizada para treinamento, teste público ou teste privado (detalhes mais adiante).
df = pd.read_csv(PATH/'fer2013.csv')
df.head()

In [ ]:
# As expressões das imagens na descrição do desafio do Kaggle, de 0 a 6 são
# Raiva, Nojo, Medo, Alegria, Tristeza, Surpresa, Neutro
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
for l in range(len(labels)):
    print(f'{l} = {labels[l]}')

In [ ]:
# Também temos a utilização das imagens, treinamento, teste público e teste privado 
# Os desafios Kaggle possuem um teste público, com a base que disponibilizam para criação dos modelos
# e uma privada que somente é testada com a submissão dos resultados
usages = df['Usage'].unique();usages

In [ ]:
# Vamos separar os data frames em treinamento, teste público e teste privado
df_train = df[df['Usage'] == 'Training']
df_valid = df[df['Usage'] == 'PublicTest']
df_test = df[df['Usage'] == 'PrivateTest']

# Os tamanhos de cada data frame
n_train = len(df_train)
n_valid = len(df_valid)
n_test = len(df_test)
n = len(df)

print(f'{n_train} (Training) + {n_valid} (PublicTest) + {n_test} (PrivateTest) = {n} images')

In [ ]:
# histograma das bases de treinamento, teste público e teste privado
def setup_axe(axe,df,title):
    axe.hist(df['emotion'])
    axe.set_xticks(list(range(len(labels))))
    axe.set_xticklabels(labels, rotation=45)
    axe.set_title(title)

fig, axes = plt.subplots(1,3, figsize=(15,5))
setup_axe(axes[0],df_train,'treinamento')
setup_axe(axes[1],df_valid,'validação')
setup_axe(axes[2],df_test,'teste')
plt.show()

In [ ]:
# Vamos avaliar a proporção das imagens na base de treinamento de cada uma das 7 expressões
ne = df_train['emotion'].value_counts(ascending=True)
for k,v in zip(ne.keys(),ne.values):
    pct = round(v/n*100,2)
    print(f'({pct}%) {v} {labels[k]}')

In [ ]:
# A função abaixo recria a imagem a partir dos pixels. Lembrando que as imagens são sequências de 3 números para cada pixel, ex (48,48,3)
# Vamos representar a expressão como um inteiro e os pixels de uma imagem como um array numpy 
def row2image(row):
    pixels, emotion = row['pixels'], row['emotion']
    img = np.array(pixels.split())
    img = img.reshape(48,48)
    image = np.zeros((48,48,3))
    image[:,:,0] = img
    image[:,:,1] = img
    image[:,:,2] = img
    return np.array([image.astype(np.uint8), emotion])

# Apresenta imagem já convertida com o título da expressão
def show(img_title):
    plt.imshow(img_title[0])
    plt.title(labels[img_title[1]])

In [ ]:
# Vamos dar uma olhada na base para ver as imagens? Brinquem à vontade, aqui não somos Gilberto Gil, mas vamos de 2222
row = df_train.iloc[2222]
img = row2image(row)
show(img)

# Armazenar os dados (pixels) em forma de imagens nas pastas train/val/test

In [ ]:
# Com a função de pixels (com 3 números cada) para arquivos, vamos agora criar as pastas para armazenar cria as pastas train, valid e test
Path(PATH/'train').mkdir(exist_ok=True)
Path(PATH/'valid').mkdir(exist_ok=True)
Path(PATH/'test').mkdir(exist_ok=True)

In [ ]:
# Define as variáveis das pastas de treinamento, teste público e teste privado
PATH_train = PATH/'train'
PATH_valid = PATH/'valid'
PATH_test = PATH/'test'

In [ ]:
# cria as 7 pastas das expressões em cada uma das pastas train, train, val e test
for l in labels:
    Path(PATH_train/l).mkdir(exist_ok=True)
    Path(PATH_valid/l).mkdir(exist_ok=True)
    Path(PATH_test/l).mkdir(exist_ok=True)

In [ ]:
# Cria as imagens a partir dos pixels e salva nas subpastas correspondentes. Aqui utilizaremos o teste público como validação
# e teste privado como teste
def createImages(df,path):
    for e in range(len(labels)):
        df_e = df[df['emotion'] == e]
        path_e = path/labels[e]
        i=0
        for index, row in df_e.iterrows():
            img = row2image(row)
            image = PIL.Image.fromarray(img[0], 'RGB')
            fname = str(e)+'_'+str(i)+'.jpg'
            image.save(path_e/fname)
            i += 1

In [ ]:
# Calcula o tempo para criação das imagens e posicionamento nas subpastas
%%time
createImages(df_train,PATH_train)
createImages(df_valid,PATH_valid)
createImages(df_test,PATH_test)

In [ ]:
# Vamos dar uma olhada se a criação ocorreu da forma desejada
p = PATH_train/'Angry'
p.ls()[:5]

# Criar o ImageDataBunch parametrizado para resnet50

In [ ]:
# Cria transformações para as imagens como forma de aumentar o dataset  data augmentation
tfms = get_transforms()

In [ ]:
# ImageDataBunch
# bs=16, size=299 : ajustamos os parametros para o laboratório terminar num tempo razoável, mas o melhor seria o valor 
# do hiperparâmetro size=299 para ajudar à resnet50
data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, bs=16, size=224)
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

# Treinar o modelo com transfer learning (resnet50)

In [ ]:
# cria o learner a partir do resnet50 com métrica de acurácia para buscar dos coeficientes da rede neural
learn = cnn_learner(data, models.resnet50, metrics=[accuracy])

In [ ]:
# one cycle training com 4 épocas
learn.fit_one_cycle(4)

In [ ]:
# salvando o estágio atual (caso o preemptive resolve nos ajudar O.o)
learn.save('fer2013-stage-1')

In [ ]:
learn.load('fer2013-stage-1')

In [ ]:
# Com o modelo treinado somente com as novas imagens, vamos "descongelar" o learner para reaprender com o resnet50 completo
learn.unfreeze()

In [ ]:
# buscar os melhores learning rates
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
# com base no gráfico vamos retreinar com 6 épocas e learning rates específicos
learn.fit_one_cycle(6, max_lr=slice(1e-6,1e-4))

In [ ]:
# salvar estágio atual
learn.save('fer2013-stage-2')

# exportar o modelo treinado. Importante para implantarmos no nosso aplicativo web!
learn.export('modelo.pkl')

# Resultados

Com o modelo treinado, vamos avaliar os resultados, principalmente com relação a acurácia.

In [ ]:
learn.load('fer2013-stage-2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# A matriz de confusão mostra que, para as expressões com mais dados para treinamento temos um acerto maior
interp.plot_confusion_matrix()

In [ ]:
# Avaliando quais foram as piores classificações em termos de erro
interp.plot_top_losses(9, figsize=(15,11))

# Acurácia

In [ ]:
val_probs, val_targets = learn.get_preds()

In [ ]:
val_preds = torch.argmax(val_probs,1)

In [ ]:
val_acc = (val_preds == val_targets).type(torch.FloatTensor).mean().item()
print(f'valid accuracy: {round(val_acc*100,2)}%')

Apesar de parecer pouco, um modelo com 64% de acurácia estaria bem colocado no desafio do Kaggle, isso sem muito esforço, pois poderíamos melhorar o tamanho da imagem e realizar limpeza das imagens expúreas/dúbias.


# Utilizando o modelo em produção (predição de novas fotos)

Agora vamos testar nosso modelo com aquelas imagens que não foram utilizadas nem no treino nem na validação. Brinquem à vontade, alterando a pasta de 'Happy' para outras expressões, bem como o índice do array de arquivos.

In [ ]:
data.classes

In [ ]:
# get image from test folder (images arquived in subfolders)
p = PATH_test/'Happy'
url = p.ls()[1]
img = open_image(url)
img

In [ ]:
# get prediction
#probs = learn.predict(img)
#prediction = learn.data.classes[probs.argmax()]
#prediction

pred_class,pred_idx,outputs = learn.predict(img)
pred_class

## Buscando imagem da internet

Vamos tentar buscar uma imagem na internet e realizar uma classificação da expressão facial? 

In [ ]:
# Download da imagem
!wget http://thesharpe.com/wp-content/uploads/2017/01/surprise-someone-1-1024x768.jpg
    
# Transformação para grayscale (as imagens treinadas e validadas também estão em grayscale)
img = PIL.Image.open('surprise-someone-1-1024x768.jpg').convert('LA')

# Salvar a imagem (percebam que o formato deve ser png)
img.save('surprise-someone-1-1024x768.png')    
    
# Mudança de semântica! Antes img era uma imagem do pacote PIL, agora é uma imagem fast.ai (open_image)    
img = open_image('./surprise-someone-1-1024x768.png')

# Vamos dar uma olhada na imagem baixada em grayscale
img

In [ ]:
# Vamos realizar as transformações utilizadas nas imagens de treino? A imagem a ser classificada será reduzida e poderá
# sofrer algumas rotações e distorções
tfms = get_transforms()

for transformation in tfms:
    img = img.apply_tfms(transformation, size=224)

img

In [ ]:
# finalmente, vamos prever o que o Kevin McCallister estava expressando
pred_class,pred_idx,outputs = learn.predict(img)
pred_class

# Modelo treinado numa aplicação web

Agora que temos um modelo treinado, funcionando razoavelmente bem, que tal criarmos uma aplicação web que recebe fotos e classifica a expressão do rosto?

